# Train Baseline

This notebook shows how to train the baseline model for this competition.

In [1]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))


import torch
import wandb

from utils.train import TrainConfig, run_train_model
from utils.augmentations import get_default_transform
from utils import creating_dataset

# this is the implementation of the custom baseline model
from utils import hvatnet

## Define trainer configuration

The `TrainConfig` class is used to train the baseline model - have a look at the parameters it has!

In [2]:
train_config = TrainConfig(exp_name='test_2_run_fedya', p_augs=0.3, batch_size=64, eval_interval=150, num_workers=0)


## Load data

In [3]:
# DATA_PATH = r"F:\Dropbox (Personal)\BCII\BCI Challenges\2024 ALVI EMG Decoding\dataset_v2_blocks\dataset_v2_blocks"
DATA_PATH = "/media/lutetia/Extreme SSD/EMG_Yun/bci-initiative-alvi-hci-challenge/dataset_v2_blocks/dataset_v2_blocks"

def count_parameters(model): 
    n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    n_total = sum(p.numel() for p in model.parameters())
    print(f"Total: {n_total/1e6:.2f}M, Trainable: {n_trainable/1e6:.2f}M")
    return n_total, n_trainable


    
## Data preparation
transform = get_default_transform(train_config.p_augs)
data_paths = dict(datasets=[DATA_PATH],
                    hand_type = ['left', 'right'], # [left, 'right']
                    human_type = ['health', 'amputant'], # [amputant, 'health']
                    test_dataset_list = ['fedya_tropin_standart_elbow_left'])
data_config = creating_dataset.DataConfig(**data_paths)
train_dataset, test_dataset = creating_dataset.get_datasets(data_config, transform=transform)



Getting val datasets
Number of moves: 72 | Dataset: fedya_tropin_standart_elbow_left
Reorder this dataset fedya_tropin_standart_elbow_left True
Getting train datasets
Number of moves: 72 | Dataset: fedya_tropin_standart_elbow_left
Reorder this dataset fedya_tropin_standart_elbow_left True
Number of moves: 70 | Dataset: valery_first_standart_elbow_left
Reorder this dataset valery_first_standart_elbow_left True
Number of moves: 135 | Dataset: alex_kovalev_standart_elbow_left
Reorder this dataset alex_kovalev_standart_elbow_left True
Number of moves: 72 | Dataset: anna_makarova_standart_elbow_left
Reorder this dataset anna_makarova_standart_elbow_left True
Number of moves: 62 | Dataset: artem_snailbox_standart_elbow_left
Reorder this dataset artem_snailbox_standart_elbow_left True
Number of moves: 144 | Dataset: matthew_antonov_standart_elbow_left
Reorder this dataset matthew_antonov_standart_elbow_left True
Number of moves: 144 | Dataset: misha_korobok_standart_elbow_left
Reorder this da

## Initialize the model
As you can see below, the model has a number of hyperparameters specifying its architecture and parameters. These are the parameters used to generate the baseline predictions.

In [4]:
model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
                            n_res_blocks=3, n_blocks_per_layer=3,
                            n_filters=128, kernel_size=3,
                            strides=(2, 2, 2), dilation=2, 
                            small_strides = (2, 2))
model = hvatnet.HVATNetv3(model_config)
count_parameters(model)

Number of parameters: 4210788
Total: 4.23M, Trainable: 4.23M


(4231429, 4231429)

Remember that the predictions are downsampled at 25Hz from the data originally recorded at 200Hz. The `hvatnet` model used here, automatically and correctly downsamples the data during predictions. Make sure that your model's oputput is also downsampled!

In [5]:
X, Y = train_dataset[0]
print(f"X shape: {X.shape}, Y shape: {Y.shape}")

Y_hat = model(torch.tensor(X).unsqueeze(0)).squeeze().detach().numpy()

print(f"Predictions shape: {Y_hat.shape}")

assert Y.shape == Y_hat.shape, "Predictions have the wrong shape!"

X shape: (8, 256), Y shape: (20, 32)
Predictions shape: (20, 32)


The following code trains the baseline model using training code defined in `utils`.

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import numpy as np
from scipy.ndimage import uniform_filter1d, gaussian_filter1d
from scipy.signal import savgol_filter

# Post-processing functions
def moving_average(data, window_size, axis=0):
    return uniform_filter1d(data, size=window_size, axis=axis, mode='reflect')

def savitzky_golay_filter(data, window_size, poly_order, axis=0):
    return savgol_filter(data, window_size, poly_order, axis=axis)

def gaussian_smoothing(data, sigma, axis=0):
    return gaussian_filter1d(data, sigma, axis=axis)

def example_post_process(data):
    return moving_average(data, window_size=10,axis=-1) #window_size=3-15
    # return savitzky_golay_filter(data, window_size=5, poly_order=2) #window_size=3-15, poly_order=2-5
    # return gaussian_smoothing(data, sigma=1) #sigma=0.1-2


run_train_model(model, (train_dataset, test_dataset), train_config, device, post_process=example_post_process)


Completed initialization of scheduler
******************************************************************************************************************************************************

overall_steps 150: 0.18349376320838928
val loss: 0.1840226948261261
saved model:  step_150_loss_0.1840.safetensors


******************************************************************************************************************************************************

overall_steps 300: 0.15588481724262238
val loss: 0.1777040958404541
saved model:  step_300_loss_0.1777.safetensors


******************************************************************************************************************************************************

overall_steps 450: 0.16865788400173187
val loss: 0.2012651115655899


******************************************************************************************************************************************************

overall_steps 600: 0.12270639091730118
val loss: 0

0.13697326183319092